<a href="https://colab.research.google.com/github/vxfontes/machineLearning/blob/main/modeloGeralML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Área geral de importação

In [ ]:
# importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# importando pandas e conectando com o google colab caso esteja sendo usado
# from google.colab import drive
# drive.mount('/content/drive') # recuperando a página

In [ ]:
# importando arquivo de analise
file_path = '/content/drive/MyDrive/Colab Notebooks/PASTA DO SEU ARQUIVO AQUI;ARQUIVO.CSV'
data = pd.read_csv(file_path) # lendo arquivo e armazenando na variável dataset

In [ ]:
#caso seja arquivo armazenado no computador
file_path = 'PASTA DO SEU ARQUIVO AQUI;ARQUIVO.CSV'
data = pd.read_csv(file_path)

## Fazendo análise da base de dados


In [ ]:
# exibindo tuplas iniciais, finais e quantidade de colunas
display(data)

In [ ]:
# visualizando informações e se temos ou não dados nulos
data.info()

In [ ]:
# Contando a quantidade de valores nulos por coluna
data.isnull().sum()

In [ ]:
# quantidade de valores por variavel
data.value_counts()

In [ ]:
# Verificando as informações estatísticas da base
data.describe()

Agora é a hora que tomamos a primeira decisão: o que fazer com os dados nulos
- caso exista uma tabela com muitos dados nulos, o melhor seria exclui-la do nosso dataset
- caso tenha pouco valor nulo, podemos obtar por apagar as linhas nulas ou atribuir a média ou mediana para esses campos

anotamos as decições nesse bloco

In [ ]:
# ver a porcentagem de dados nulos
data.isna().sum()/data.shape[0] 

In [ ]:
# visualizando dados nulos para possível manipulação
data[data.isna().any(axis=1)]

In [ ]:
# Retirando as colunas que tem muitos valores nulos
retirar = ["Coluna 1", "Coluna 2"] # colunas que queremos remover, com sensitive case
data = data.drop(retirar, axis=1)

In [ ]:
# Atribuindo a média para colunas sem valores
data.loc[data.Coluna.isnull(),'Coluna'] = data['Coluna'].mean()

In [ ]:
# Eliminando todas as outras linhas com valores vazios
data = data.dropna()

**Agora não existem mais valores nulos na nossa tabela**

In [ ]:
# Verificando o número de valores únicos em cada coluna
data.nunique()

In [ ]:
# verificando cada valor unico por coluna
for col in data.columns:
    print(f"Coluna: {col}")
    print(data[col].unique())
    print("-" * 30)

Sem valores nulos, podemos então remover as colunas que possuem dados únicos como ids, nomes pessoais, entre outros

In [ ]:
# removendo counas específicas demais
retirar = ["Coluna 1", "Coluna 2"] # colunas que queremos remover, com sensitive case
data = data.drop(retirar, axis=1)

**Sem valores nulos e sem colunas inúteis, podemos procurar por mais colunas para tratar**

In [ ]:
# tipos de colunas
data.dtypes
data.info()

In [ ]:
# gráficos demonstrando quantidade de valores iguais em todas as colunas 

num_lst = []
cat_lst = []

from pandas.api.types import is_string_dtype, is_numeric_dtype

for column in data:
    plt.figure(column, figsize = (5,5))
    plt.title(column)
    if is_numeric_dtype(data[column]):
        data[column].plot(kind = 'hist')
        num_lst.append(column)
    elif is_string_dtype(data[column]):
        data[column].value_counts().plot(kind = 'bar')
        cat_lst.append(column)
        
print(num_lst)
print(cat_lst)

Podemos agora verificar se nossa base de dados possui valores duplicados, pois precisamos removê-las
Essa decisão melhora muito nosso modelo ML, pois dá dados únicos para cálculo

In [ ]:
data.duplicated()
data.duplicated().sum() # verificando quantidade de tuplas duplicadas

In [ ]:
data.drop_duplicates(inplace=True) # removendo tuplas duplicadas

Agora podemos tratar da seguinte forma
- caso tenham apenas 2 valores em uma coluna do tipo object, podemos utilizar lambda em uma coluna nova ou na mesma coluna para afirmar ou negar
- caso tenha mais de dois valores, podemos utilizar uma tecnica chamada LabelEncoder
- caso não exista uma relação entre os valores (um valor maior que o outro), vamos usar o One Hot Encoding ou get dummies
- Para isso, podemos usar o OneHotEncoder do Scikit-Learn:
    - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
- Ou o próprio get_dummies do pandas:
    - https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

ambos retornam matrizes que precisamos concatenar


In [ ]:
# usando lambda
data.Coluna.value_counts() ## vendo quais valores existem naquela coluna especifica
data["Coluna nova"] = data.gender.apply(lambda x:1 if x == 'Valor positivo' else 0)
data = data.drop(["Coluna velha"], axis=1) ## removendo coluna velha

In [ ]:
# usando metodo label para transformar colunas
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder() # criando label
data["Coluna nova"] = encoder.fit_transform(data.Coluna) # criando coluna nova fazendo a transformação da velha
data.groupby(["Coluna nova", "Coluna velha"])["Coluna velha"].count() # agrupando colunas para descobrir quais valores são quais números

In [ ]:
# usando get_dummies
pd.get_dummies(data.Coluna)

# Unindo essas informações no dataset
data = pd.concat([data,pd.get_dummies(data.Coluna)],axis=1)
## concatenamos então uma matriz com valores que representam individualmente cada caso da antiga coluna, isso fara com que a coluna seja substituida

## Análise Exploratória
- Vamos tentar visualizar como as informações de cada item estão distribuídas
- Vamos ver a correlação entre cada um dos itens

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(data.corr(), annot=True, cmap="Wistia")
plt.show()

# outra forma de ver a mesma análise
# sns.pairplot(tabela)
# plt.show()

# Treino X Teste

#### Com isso, podemos partir para a preparação dos dados para treinarmos o Modelo de Machine Learning

- Separando em dados de treino e dados de teste

In [ ]:
from sklearn.model_selection import train_test_split

# separamos primeiramente qual coluna será a que queremos prever e qual será parte da previsão
x = data.drop("ColunaQueSeraPrevista", axis=1)
y = data["ColunaQueSeraPrevista"]

xTreino, xTeste, yTreino, yTeste = train_test_split(x, y, test_size=0.3, random_state=1)

# Problema de classificação
### Temos 2 principais algoritmos para classificação de itens
- Árvore de decisão
- KNN

In [ ]:
# Importando os algoritmos e criando os classificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

neigh = KNeighborsClassifier(n_neighbors=3)
clfArvore = tree.DecisionTreeClassifier(random_state=0)

In [ ]:
# dando o fit nos modelos, ou seja, inserindo nossos dados de treino
neight = neigh.fit(xTreino, yTreino)
clfArvore = clfArvore.fit(xTreino, yTreino)

**Podemos então fazer a predição dos dados com os dados de teste para avaliar os resultados**

In [ ]:
# Fazendo a predicão
predKNN = neigh.predict(xTeste)
predArvore = clfArvore.predict(xTeste)

#### Visualizando a matriz de confusão do código, onde teremos
[verdadeiro Negativo, Falso positivo]
[falso negativo, verdadeiro positivo]

In [ ]:
# matriz de confusão para o KNN
from sklearn.metrics import confusio_matrix

confusion_matrix(yTeste, predKNN)

In [ ]:
# matriz de confusão para o KNN
from sklearn.metrics import confusio_matrix

confusion_matrix(yTeste, predArvore)

#### Para avaliar o modelo, usaremos algumas métricas
- acurácia: quantos valores foram acertados, seja positivo ou negativo. quantos acertos com relação ao total em %
- precisão: relação entre o falso positivo e o verdadeiro positivo. dos valores positivos, quantos estavam corretos
- recall: relação entre os falsos negativos  e verdadeiros positivos. dos valores reais verdadeiros, quantos conseguimos encontrar

In [ ]:
# Importando as métricas de avaliação
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
# Acurácia
print(accuracy_score(yTeste,predKNN))
print(accuracy_score(yTeste,predArvore))

In [ ]:
# Precisão
print(precision_score(yTeste,predKNN))
print(precision_score(yTeste,predArvore))

In [ ]:
# Recall
print(recall_score(yTeste,predKNN))
print(recall_score(yTeste,predArvore))

# Problema de regressão
### Temos 3 importantes algoritmos de regressão
- Regressão linear
- Ridge
- Lasso

In [ ]:
# Importando os algoritmos e criando os classificadores
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso

ridReg = Ridge(alpha=0.5)
clfLog = LogisticRegression(random_state=0, max_iter=1000)
lassoReg = Lasso(alpha=0.1)

In [ ]:
# fazendo fit dos modelos acima
ridReg = ridReg.fit(xTreino, yTreino)
clfLog = clfLog.fit(xTreino, yTreino)
lassoReg = lassoReg.fit(xTreino, yTreino)

In [ ]:
# Fazendo a predicão
predRidge = ridReg.predict(xTeste)
predLasso = lassoReg.predict(xTeste)
predLog = clfLog.predict(xTeste)

### Com a previsão feita com nosso algoritmo de teste, podemos verificar nossa regressão de duas formas, com o r² e com análise gráfica

In [ ]:
# Importando as métricas de avaliação
from sklearn import metrics ## avaliação em %

print(metrics.r2_score(yTeste, pred_Ridge))
print(metrics.r2_score(yTeste, pred_Lasso))
print(metrics.r2_score(yTeste, pred_Log))

**Usando gráfico de análise com tabela auxiliar**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

tabela_auxiliar = pd.DataFrame()
tabela_auxiliar["y_teste"] = yTeste
tabela_auxiliar["Previsoes Ridge"] = predRidge
tabela_auxiliar["Previsoes Regressao Linear"] = predLog
tabela_auxiliar["Previsoes Lasso"] = predLasso

plt.figure(figsize=(15,6))
sns.lineplot(data=tabela_auxiliar)
plt.show()

observar qual curva se aproxima mais a curva de teste

# Prevendo dados
- utilizando as colunas do XTreino, criamos um dataframe com valores para obsevarmos qual a predição

In [ ]:
dataset = {'Coluna1': [valorLinha1, valorLinha2],
        'Coluna2': [valorLinha1, valorLinha2]}

tabela = pd.DataFrame(dataset)

## para cada algoritmo que treinamos
previsao = lassoReg.predict(tabela)

# valor previsto
print(previsao)